# Main 

Zexuan: The main file for category ML model production

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sqlite3
from sqlalchemy.engine import create_engine

from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from tpot import TPOTClassifier

import tpot_functions 
import periods 
import utils


Define some key variables and numbers

In [26]:
target = 'deal_probability'
LIMIT = None
TIMEOUT_MINS = 30
#SCORING = 'neg_mean_squared_error'
RS = 27

In [27]:
train = pd.read_csv('data/train.csv', parse_dates=['activation_date'], infer_datetime_format=True)[:LIMIT]
train_pd = pd.read_csv('train_price_normalized.csv')[:LIMIT]
#train_periods = pd.read_csv('data/periods_aggregate_features_items.csv')

In [28]:
list(train)

['item_id',
 'user_id',
 'region',
 'city',
 'parent_category_name',
 'category_name',
 'param_1',
 'param_2',
 'param_3',
 'title',
 'description',
 'price',
 'item_seq_number',
 'activation_date',
 'user_type',
 'image',
 'image_top_1',
 'deal_probability']

In [29]:
list(train_pd)

['Unnamed: 0',
 'item_id',
 'category_name',
 'price',
 'deal_probability',
 'cat_price',
 'cat_std',
 'norm_price',
 'cat_price_log',
 'cat_std_log',
 'norm_price_log',
 'price_log']

In [30]:
train_yes = train[['description','param_1','param_2','param_3','image','image_top_1']].isnull().astype(int)
#train_yes = train_yes.rename(index=str, columns={"description": "if_description", "param_1": "if_param_1"}
train_yes.head()

,description,param_1,param_2,param_3,image,image_top_1
0,0,0,1,1,0,0
1,0,0,1,1,0,0
2,0,0,1,1,0,0
3,0,0,1,1,0,0
4,0,0,0,0,0,0


In [31]:
#city = preprocessing.LabelEncoder()
#city.fit(train['city'])  
#list(city.classes_)
#train['city'] = city.transform(train['city'])

#userid = preprocessing.LabelEncoder()
#userid.fit(train['user_id'])  
#list(userid.classes_)
#train['user_id'] = userid.transform(train['user_id'])

In [32]:
train_pd = pd.concat([train_pd[['item_id', 'norm_price_log',target]], train_yes], axis=1)
train = pd.concat([train_pd,pd.get_dummies(train[['user_type', 'parent_category_name', 'category_name', 'region',
                                                  'city', 'item_seq_number','activation_date']])], axis=1)      
  

In [33]:
list(train)

['item_id',
 'norm_price_log',
 'deal_probability',
 'description',
 'param_1',
 'param_2',
 'param_3',
 'image',
 'image_top_1',
 'item_seq_number',
 'activation_date',
 'user_type_Company',
 'user_type_Private',
 'user_type_Shop',
 'parent_category_name_Бытовая электроника',
 'parent_category_name_Для бизнеса',
 'parent_category_name_Для дома и дачи',
 'parent_category_name_Животные',
 'parent_category_name_Личные вещи',
 'parent_category_name_Недвижимость',
 'parent_category_name_Транспорт',
 'parent_category_name_Услуги',
 'parent_category_name_Хобби и отдых',
 'category_name_Автомобили',
 'category_name_Аквариум',
 'category_name_Аудио и видео',
 'category_name_Билеты и путешествия',
 'category_name_Бытовая техника',
 'category_name_Велосипеды',
 'category_name_Водный транспорт',
 'category_name_Гаражи и машиноместа',
 'category_name_Готовый бизнес',
 'category_name_Грузовики и спецтехника',
 'category_name_Детская одежда и обувь',
 'category_name_Дома, дачи, коттеджи',
 'category

In [34]:
#train_new = pd.merge(train, train_periods, on='item_id')
#train_new
len(list(train))

1831

In [35]:
train = train.sort_values(by='activation_date').drop('activation_date', axis=1).set_index('item_id')

In [36]:
train_nonan = train.fillna(train.mean())

KeyboardInterrupt: 

In [ ]:
train_nonan.head()

In [37]:
len(list(train))

1829

In [ ]:
train = train.dropna()
train.head()

In [13]:
# SIMPLE: train_test_split
#X = (train.drop(['item_id',target], axis=1)).values
#y = train[target].values

#X_train, X_test, y_train, y_test = train_test_split(X, y,
#                                                    train_size=0.75, test_size=0.25)

In [12]:
# TimeSeriesSplit
X = (train.drop(target, axis=1)).values
y = train[target].values
#tss = TimeSeriesSplit(n_splits=4)
#train_index, test_index = list(tss.split(X))[-1]
#X_train, X_test = X[train_index], X[test_index]
#y_train, y_test = y[train_index], y[test_index]

Try RandomForestRegressor model 

In [15]:
config_dict = {'sklearn.ensemble.GradientBoostingRegressor': {
        'n_estimators': [100, 200, 400],
        'loss': ["ls", "lad", "huber", "quantile"],
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'max_depth': range(1, 11),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21),
        'subsample': np.arange(0.05, 1.01, 0.05),
        'max_features': np.arange(0.05, 1.01, 0.05),
        'alpha': [0.75, 0.8, 0.85, 0.9, 0.95, 0.99]
    },
        'sklearn.ensemble.RandomForestRegressor': {
        'n_estimators': [100, 200, 400],
        'max_features': np.arange(0.05, 1.01, 0.05),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21),
        'bootstrap': [True, False]
    },
              }

#RFR = RandomForestRegressor()

RFR = RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
           max_features=0.2, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=20, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=4,
           oob_score=False, random_state=None, verbose=2, warm_start=False)

#GBR  = GradientBoostingRegressor(**params)
#RSCV = RandomizedSearchCV(estimator=GBR, random_state=RS,
#                          param_distributions=config_dict['sklearn.ensemble.RandomForestRegressor'],
#                         n_iter=5,
#                         scoring='r2',
#                         verbose=2)

In [16]:
RFR.fit(X_train, y_train)

building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200


[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed: 38.5min


building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200
building tree 44 of 200
building tree 45 of 200
building tree 46 of 200
building tree 47 of 200
building tree 48 of 200
building tree 49 of 200
building tree 50 of 200
building tree 51 of 200
building tree 52 of 200
building tree 53 of 200
building tree 54 of 200
building tree 55 of 200
building tree 56 of 200
building tree 57 of 200
building tree 58 of 200
building tree 59 of 200
building tree 60 of 200
building tree 61 of 200
building tree 62 of 200
building tree 63 of 200
building tree 64 of 200
building tree 65 of 200
building tree 66 of 200
building tree 67 of 200
building tree 68 of 200
building tree 69 of 200
building tree 70 of 200
building tree 71 of 200
building tree 72 of 200
building tree 73 of 200
building tree 74 of 200
building tree 75 of 200
building tree 76 of 200
building tree 77 of 200
building tree 78

[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed: 169.6min


building tree 159 of 200
building tree 160 of 200
building tree 161 of 200
building tree 162 of 200
building tree 163 of 200
building tree 164 of 200
building tree 165 of 200
building tree 166 of 200
building tree 167 of 200
building tree 168 of 200
building tree 169 of 200
building tree 170 of 200
building tree 171 of 200
building tree 172 of 200
building tree 173 of 200
building tree 174 of 200
building tree 175 of 200
building tree 176 of 200
building tree 177 of 200
building tree 178 of 200
building tree 179 of 200
building tree 180 of 200
building tree 181 of 200
building tree 182 of 200
building tree 183 of 200
building tree 184 of 200
building tree 185 of 200
building tree 186 of 200
building tree 187 of 200
building tree 188 of 200
building tree 189 of 200
building tree 190 of 200
building tree 191 of 200
building tree 192 of 200
building tree 193 of 200
building tree 194 of 200
building tree 195 of 200
building tree 196 of 200
building tree 197 of 200
building tree 198 of 200


[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed: 218.4min finished


RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
           max_features=0.2, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=20, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=4,
           oob_score=False, random_state=None, verbose=2, warm_start=False)

Try GradientBoostingRegressor model 

In [17]:
#params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
#          'learning_rate': 0.01, 'loss': 'ls'}
#config_dict = {'sklearn.ensemble.GradientBoostingRegressor': {
#        'n_estimators': [100, 200, 400],
#        'loss': ["ls", "lad", "huber", "quantile"],
#        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
#        'max_depth': range(1, 11),
#        'min_samples_split': range(2, 21),
#        'min_samples_leaf': range(1, 21),
#        'subsample': np.arange(0.05, 1.01, 0.05),
#        'max_features': np.arange(0.05, 1.01, 0.05),
#        'alpha': [0.75, 0.8, 0.85, 0.9, 0.95, 0.99]
#    }}

#GBR  = GradientBoostingRegressor(**params)
#RSCV = RandomizedSearchCV(estimator=GBR, random_state=RS,
#                          param_distributions=config_dict['sklearn.ensemble.GradientBoostingRegressor'],
#                         n_iter=5,
#                         scoring='r2',
#                         verbose=2)


In [18]:
#RSCV.fit(X_train, y_train)

In [21]:
mse = mean_squared_error(y_test, RFR.predict(X_test))
print("MSE: %.4f" % mse)

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    4.4s


MSE: 0.0540


[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    5.8s finished


In [22]:
# Does not make sense
r2 = r2_score(y_test, RFR.predict(X_test))
print("R2: %.4f" % r2)

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    4.3s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    5.5s finished


R2: 0.1799


In [15]:
import pickle
filename = 'optimized_tabular_model.sav'
RFR = pickle.load(open(filename, 'rb'))

In [13]:
X = (train_nonan.drop(target, axis=1)).values
y = train_nonan[target].values

In [16]:
y_train = RFR.predict(X)

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    6.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:   25.5s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:   31.9s finished


In [21]:
d = {'item_id': train_nonan.index, 'deal_probability': y_train}
train_output = pd.DataFrame(data=d)
train_output.to_csv('predictions/tabular_train_new1.csv', index=False)

In [2]:
import pickle
#filename = 'optimized_tabular_model.sav'
#pickle.dump(RFR, open(filename, 'wb'))

Load test.csv file and compute 

In [3]:
import pickle
filename = 'optimized_tabular_model.sav'
RFR = pickle.load(open(filename, 'rb'))

In [38]:
test_pd = pd.read_csv('test_price_normalized.csv')[:LIMIT]
test = pd.read_csv('data/test.csv', parse_dates=['activation_date'], infer_datetime_format=True)[:LIMIT]

In [39]:
len(test)

508438

In [40]:
len(test_pd)

508438

In [41]:
test_yes = test[['description','param_1','param_2','param_3','image','image_top_1']].isnull().astype(int)
test_yes.head()

,description,param_1,param_2,param_3,image,image_top_1
0,0,0,0,0,0,0
1,0,0,1,1,1,1
2,0,0,1,1,0,0
3,0,0,0,1,1,1
4,0,0,1,1,0,0


In [42]:
len(test_yes)

508438

In [43]:
test_pd = pd.concat([test_pd[['item_id', 'norm_price_log']], test_yes], axis=1)
test = pd.concat([test_pd,pd.get_dummies(test[['user_type', 'parent_category_name', 'category_name', 'region',
                                                  'city', 'item_seq_number','activation_date']])], axis=1) 
  
test.head()

,item_id,norm_price_log,description,param_1,param_2,param_3,image,image_top_1,item_seq_number,activation_date,...,city_Яровое,city_Ярославль,city_Ярославская,city_Ясенская,city_Ясная Поляна,city_Ясногорск,city_Ясногорский,city_Ясный,city_Яшкино,city_Яя
0,6544e41a8817,NaN,0,0,0,0,0,0,66,2017-04-18,...,0,0,0,0,0,0,0,0,0,0
1,65b9484d670f,-0.080554,0,0,1,1,1,1,4,2017-04-16,...,0,0,0,0,0,0,0,0,0,0
2,8bab230b2ecd,1.163858,0,0,1,1,0,0,15,2017-04-17,...,0,0,0,0,0,0,0,0,0,0
3,8e348601fefc,0.377101,0,0,0,1,1,1,70,2017-04-17,...,0,0,0,0,0,0,0,0,0,0
4,8bd2fe400b89,0.901667,0,0,1,1,0,0,15,2017-04-15,...,0,0,0,0,0,0,0,0,0,0


In [44]:
len(test)

508438

In [45]:
test = test.set_index('item_id')

In [46]:
test.head()

,norm_price_log,description,param_1,param_2,param_3,image,image_top_1,item_seq_number,activation_date,user_type_Company,...,city_Яровое,city_Ярославль,city_Ярославская,city_Ясенская,city_Ясная Поляна,city_Ясногорск,city_Ясногорский,city_Ясный,city_Яшкино,city_Яя
item_id,,,,,,,,,,,,,,,,,,,,,
6544e41a8817,NaN,0,0,0,0,0,0,66,2017-04-18,0,...,0,0,0,0,0,0,0,0,0,0
65b9484d670f,-0.080554,0,0,1,1,1,1,4,2017-04-16,0,...,0,0,0,0,0,0,0,0,0,0
8bab230b2ecd,1.163858,0,0,1,1,0,0,15,2017-04-17,0,...,0,0,0,0,0,0,0,0,0,0
8e348601fefc,0.377101,0,0,0,1,1,1,70,2017-04-17,0,...,0,0,0,0,0,0,0,0,0,0
8bd2fe400b89,0.901667,0,0,1,1,0,0,15,2017-04-15,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
test_nonan = test.fillna(test.mean())

In [48]:
test_nonan.head()

,norm_price_log,description,param_1,param_2,param_3,image,image_top_1,item_seq_number,activation_date,user_type_Company,...,city_Яровое,city_Ярославль,city_Ярославская,city_Ясенская,city_Ясная Поляна,city_Ясногорск,city_Ясногорский,city_Ясный,city_Яшкино,city_Яя
item_id,,,,,,,,,,,,,,,,,,,,,
6544e41a8817,0.070350,0,0,0,0,0,0,66,2017-04-18,0,...,0,0,0,0,0,0,0,0,0,0
65b9484d670f,-0.080554,0,0,1,1,1,1,4,2017-04-16,0,...,0,0,0,0,0,0,0,0,0,0
8bab230b2ecd,1.163858,0,0,1,1,0,0,15,2017-04-17,0,...,0,0,0,0,0,0,0,0,0,0
8e348601fefc,0.377101,0,0,0,1,1,1,70,2017-04-17,0,...,0,0,0,0,0,0,0,0,0,0
8bd2fe400b89,0.901667,0,0,1,1,0,0,15,2017-04-15,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
#test_consistent = pd.DataFrame()
#for feat in train.columns.values:
#    if feat not in test:
#        test_consistent.loc[:, feat] = [0.0]*len(test)
#    else:
#        test_consistent.loc[:, feat] = test[feat].values

In [50]:
#test_consistent.head()

In [51]:
#test_consistent = test_consistent.set_index(test.index)

In [52]:
#len(test_consistent)

In [53]:
#test_consistent.head()

In [54]:
#test_nonan = test.fillna(test.mean())

In [55]:
#test_nonan = test_nonan.set_index()
len(test_nonan.columns)

1740

In [56]:
test_consistent_nonan = pd.DataFrame()
for feat in train.columns.values:
    if feat not in test:
        test_consistent_nonan.loc[:, feat] = [0.0]*len(test)
    else:
        test_consistent_nonan.loc[:, feat] = test_nonan[feat].values

In [57]:
test_consistent_nonan.head()

,norm_price_log,deal_probability,description,param_1,param_2,param_3,image,image_top_1,item_seq_number,user_type_Company,...,city_Яровое,city_Ярославль,city_Ярославская,city_Ясенская,city_Ясная Поляна,city_Ясногорск,city_Ясногорский,city_Ясный,city_Яшкино,city_Яя
0,0.070350,0.0,0,0,0,0,0,0,66,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.080554,0.0,0,0,1,1,1,1,4,0,...,0,0,0,0,0,0,0,0,0,0
2,1.163858,0.0,0,0,1,1,0,0,15,0,...,0,0,0,0,0,0,0,0,0,0
3,0.377101,0.0,0,0,0,1,1,1,70,0,...,0,0,0,0,0,0,0,0,0,0
4,0.901667,0.0,0,0,1,1,0,0,15,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
len(test_consistent_nonan)

508438

In [59]:
len(list(test_consistent_nonan))

1829

In [60]:
test_consistent_nonan = test_consistent_nonan.set_index(test.index)

In [61]:
test_consistent_nonan.head()

,norm_price_log,deal_probability,description,param_1,param_2,param_3,image,image_top_1,item_seq_number,user_type_Company,...,city_Яровое,city_Ярославль,city_Ярославская,city_Ясенская,city_Ясная Поляна,city_Ясногорск,city_Ясногорский,city_Ясный,city_Яшкино,city_Яя
item_id,,,,,,,,,,,,,,,,,,,,,
6544e41a8817,0.070350,0.0,0,0,0,0,0,0,66,0,...,0,0,0,0,0,0,0,0,0,0
65b9484d670f,-0.080554,0.0,0,0,1,1,1,1,4,0,...,0,0,0,0,0,0,0,0,0,0
8bab230b2ecd,1.163858,0.0,0,0,1,1,0,0,15,0,...,0,0,0,0,0,0,0,0,0,0
8e348601fefc,0.377101,0.0,0,0,0,1,1,1,70,0,...,0,0,0,0,0,0,0,0,0,0
8bd2fe400b89,0.901667,0.0,0,0,1,1,0,0,15,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
len(list(test_consistent_nonan))

1829

In [34]:
#test_consistent = test_consistent.dropna()
#test_consistent_id = test_consistent.index
#test_consistent = test_consistent.drop(['deal_probability'], axis=1)

#test_consistent['deal_probability'] = RFR.predict(test_consistent)


[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    1.6s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    7.7s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    9.9s finished


In [64]:
test_consistent_nonan_id = test_consistent_nonan.index
test_consistent_nonan = test_consistent_nonan.drop(['deal_probability'], axis=1)

test_consistent_nonan['deal_probability'] = RFR.predict(test_consistent_nonan)

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    9.1s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:   11.8s finished


In [65]:
d = {'item_id': test_consistent_nonan_id, 'deal_probability': test_consistent_nonan['deal_probability']}
test_output = pd.DataFrame(data=d)
test_output

,item_id,deal_probability
item_id,,
6544e41a8817,6544e41a8817,0.056549
65b9484d670f,65b9484d670f,0.138432
8bab230b2ecd,8bab230b2ecd,0.190808
8e348601fefc,8e348601fefc,0.246890
8bd2fe400b89,8bd2fe400b89,0.230973
c63dbd6c657f,c63dbd6c657f,0.220801
6d1a410df86e,6d1a410df86e,0.093939
e8d3e7922b80,e8d3e7922b80,0.025686
2bc1ab208462,2bc1ab208462,0.015233


In [66]:
test_output.to_csv('predictions/tabular_new.csv',index=False)